In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_ollama import ChatOllama

# ../src/utils.py
from src.utils import (
    pdf_to_vector_store,
    assign_score_with_llm_as_a_judge,
    filter_retrieved_results_by_score
)

In [2]:
llm = ChatOllama(model="llama3.2", temperature=0.1)

## Create Vector Store for Retrieval

In [ ]:
faiss_vector_store = pdf_to_vector_store(
        pdf_path = r"data\indian_constitution_text.pdf",
        chunk_size = 2000,
        chunk_overlap = 400,
        embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
)

query_retriever = faiss_vector_store.as_retriever(search_kwargs={"k": 3})

In [10]:
query = "What is the preamble of the Indian Constitution?"

results = query_retriever.invoke(query)

for result in results:
    print(result.page_content)
    print("-"*100)

THE CONSTITUTION OF INDIA PREAMB LEWE, THE PEOPLE OF INDIA, having solemnly resolved to constitute India into a 1[SOVEREIGN SOCIALIST SECULAR DEMOCRATIC REPUBLIC] and to secure to all its citizens:JUSTICE, social, economic and political; LIBERTY of thought, expression, belief, faith and worship;EQUALITY of status and of opportunity;and to promote among them allFRATERNITY assuring the dignity of the individual and the 2[unity and integrity of the Nation];IN OUR CONSTITUENT ASSEMBLY this twenty-sixth day of November, 1949, do HEREBY ADOPT, ENACT AND GIVE TO OURSELVES THIS CONSTITUTION.______________________________________________1. Subs. by the Constitution (Forty-second Amendment) Act, 1976, s.2, for "SOVEREIGN DEMOCRATIC REPUBLIC" (w.e.f. 3-1-1977).2. Subs. by s. 2, ibid., for "Unity of the Nation" (w.e.f. 3-1-1977).
----------------------------------------------------------------------------------------------------
PREFACEThis is the  sixth pocket size edition of the Constitution of 

In [11]:
for result in results:
    score = assign_score_with_llm_as_a_judge(
        query = query,
        result = result.page_content,
        llm = llm
    )

    print(f"Score for the document: {score['score']}")
    print(result.page_content)
    print("-"*100)

Score for the document: 5
THE CONSTITUTION OF INDIA PREAMB LEWE, THE PEOPLE OF INDIA, having solemnly resolved to constitute India into a 1[SOVEREIGN SOCIALIST SECULAR DEMOCRATIC REPUBLIC] and to secure to all its citizens:JUSTICE, social, economic and political; LIBERTY of thought, expression, belief, faith and worship;EQUALITY of status and of opportunity;and to promote among them allFRATERNITY assuring the dignity of the individual and the 2[unity and integrity of the Nation];IN OUR CONSTITUENT ASSEMBLY this twenty-sixth day of November, 1949, do HEREBY ADOPT, ENACT AND GIVE TO OURSELVES THIS CONSTITUTION.______________________________________________1. Subs. by the Constitution (Forty-second Amendment) Act, 1976, s.2, for "SOVEREIGN DEMOCRATIC REPUBLIC" (w.e.f. 3-1-1977).2. Subs. by s. 2, ibid., for "Unity of the Nation" (w.e.f. 3-1-1977).
----------------------------------------------------------------------------------------------------
Score for the document: 1
PREFACEThis is th

## Response Generation

In [14]:
rag_prompt = PromptTemplate.from_template(
    """
    You are an intelligent assistant that generates a response based on the provided context.

    Context: {context}
    Query: {query}

    - Generate a short, concise and informative response based on the context and query.
    - Ensure that your response is relevant to the query and supported by the context.
    - If no relevant information is found in the context, respond with "I don't know".
    - If the context is empty, respond with "No context provided".
    """
)

In [39]:
rewrite_query = RunnableLambda(
    lambda x: {
        **x,
        "query": llm.invoke(
            f"""
            You are a query rewriter for a retrieval system.
            The user may ask follow-up questions that refer to earlier conversation.
            Rewrite the current question so that it is fully self-contained and clear for a search engine.

            Previous question: {x.get('last_question', '')}
            Current question: {x['query']}

            Strictly respond with the rewritten query and no additional text.
            """
        ).content.strip()
        if x.get("last_question") else x["query"]
    }
)

In [40]:
rag_chain = (
    # rewrite the query to make it self-contained based on the last question asked
    rewrite_query |
    # retrieve relevant context based on the query ---> filter using llm_as_a_judge
    RunnablePassthrough.assign(
        context=lambda x: filter_retrieved_results_by_score(
            results=query_retriever.invoke(x['query']),
            query=x['query'],
            llm=llm,
            threshold=x['threshold'] if 'threshold' in x else 4
        )
    ) |
    # prepare the input for the rag prompt
    RunnableLambda(
        lambda x: {
            "context" : "\n".join(x['context']) if x['context'] else "No context provided",
            "query" : x['query']
        }
    ) |
    rag_prompt |
    llm
)

In [41]:
last_question = None

while True:
    query = input()
    print(f"Human query: {query}")

    if query.lower() == "exit":
        break

    response = rag_chain.invoke({
        "query": query,
        "last_question": last_question,
        "threshold": 4
    }).content

    last_question = query

    print("AI response:", response)
    print("\n")

Human query: What is the preamble of the Indian Constitution?
AI response: The preamble of the Indian Constitution reads:

"THE CONSTITUTION OF INDIA PREAMB LEWE, THE PEOPLE OF INDIA, having solemnly resolved to constitute India into a 1[SOVEREIGN SOCIALIST SECULAR DEMOCRATIC REPUBLIC] and to secure to all its citizens:JUSTICE, social, economic and political; LIBERTY of thought, expression, belief, faith and worship;EQUALITY of status and of opportunity;and to promote among them allFRATERNITY assuring the dignity of the individual and the 2[unity and integrity of the Nation];IN OUR CONSTITUENT ASSEMBLY this twenty-sixth day of November, 1949, do HEREBY ADOPT, ENACT AND GIVE TO OURSELVES THIS CONSTITUTION."

The preamble outlines the fundamental principles and objectives of the Indian Constitution, including justice, liberty, equality, and fraternity.


Human query: When was it enacted to the Constitution?
AI response: The preamble of the Indian Constitution was enacted on November 26, 